In [33]:
import pandas as pd
import numpy as np 
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [21]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_columns', None)

In [22]:
df = pd.read_csv("df_dummies_f.csv")
#deleting col de sexo pois so temos feminino aqui 
df = df.drop('sexo', axis=1)
df.head()

,faixa_remuneracao_media_sm,PRODUÇÃO DE LAVOURAS TEMPORÁRIAS,HORTICULTURA E PRODUTOS DE VIVEIRO,PRODUÇÃO DE LAVOURAS PERMANENTES,PECUÁRIA,"ATIVIDADES DE SERVIÇOS RELACIONADOS COM A AGRICULTURA E A PECUÁRIA, EXCETO ATIVIDADES VETERINÁRIAS","SILVICULTURA, EXPLORAÇÃO FLORESTAL E SERVIÇOS RELACIONADOS","PESCA, AQÜICULTURA E SERVIÇOS RELACIONADOS",EXTRAÇÃO DE CARVÃO MINERAL,EXTRAÇÃO DE PETRÓLEO E GÁS NATURAL,ATIVIDADES DE SERVIÇOS RELACIONADOS COM A EXTRAÇÃO DE PETRÓLEO E GÁS EXCETO A PROSPECÇÃO REALIZADA POR TERCEIROS,EXTRAÇÃO DE MINÉRIO DE FERRO,EXTRAÇÃO DE MINERAIS METÁLICOS NÃOFERROSOS,"EXTRAÇÃO DE PEDRA, AREIA E ARGILA",EXTRAÇÃO DE OUTROS MINERAIS NÃOMETÁLICOS,ABATE E PREPARAÇÃO DE PRODUTOS DE CARNE E DE PESCADO,"PROCESSAMENTO, PRESERVAÇÃO E PRODUÇÃO DE CONSERVAS DE FRUTAS, LEGUMES E OUTROS VEGETAIS",PRODUÇÃO DE ÓLEOS E GORDURAS VEGETAIS E ANIMAIS,LATICÍNIOS,"MOAGEM, FABRICAÇÃO DE PRODUTOS AMILÁCEOS E DE RAÇÕES BALANCEADAS PARA ANIMAIS",FABRICAÇÃO E REFINO DE AÇÚCAR,TORREFAÇÃO E MOAGEM DE CAFÉ,FABRICAÇÃO DE OUTROS PRODUTOS ALIMENTÍCIOS,FABRICAÇÃO DE BEBIDAS,FABRICAÇÃO DE PRODUTOS DO FUMO,FIAÇÃO,TECELAGEM INCLUSIVE FIAÇÃO E TECELAGEM,"FABRICAÇÃO DE ARTEFATOS TÊXTEIS, INCLUINDO TECELAGEM","ACABAMENTOS EM FIOS, TECIDOS E ARTIGOS TÊXTEIS, POR TERCEIROS",FABRICAÇÃO DE ARTEFATOS TÊXTEIS A PARTIR DE TECIDOS EXCETO VESTUÁRIO E DE OUTROS ARTIGOS TÊXTEIS,FABRICAÇÃO DE TECIDOS E ARTIGOS DE MALHA,CONFECÇÃO DE ARTIGOS DO VESTUÁRIO,FABRICAÇÃO DE ACESSÓRIOS DO VESTUÁRIO E DE SEGURANÇA PROFISSIONAL,CURTIMENTO E OUTRAS PREPARAÇÕES DE COURO,FABRICAÇÃO DE ARTIGOS PARA VIAGEM E DE ARTEFATOS DIVERSOS DE COURO,FABRICAÇÃO DE CALÇADOS,DESDOBRAMENTO DE MADEIRA,"FABRICAÇÃO DE PRODUTOS DE MADEIRA, CORTIÇA E MATERIAL TRANÇADO EXCETO MÓVEIS",FABRICAÇÃO DE CELULOSE E OUTRAS PASTAS PARA A FABRICAÇÃO DE PAPEL,"FABRICAÇÃO DE PAPEL, PAPELÃO LISO, CARTOLINA E CARTÃO",FABRICAÇÃO DE EMBALAGENS DE PAPEL OU PAPELÃO,"FABRICAÇÃO DE ARTEFATOS DIVERSOS DE PAPEL, PAPELÃO, CARTOLINA E CARTÃO",EDIÇÃO,IMPRESSÃO E SERVIÇOS CONEXOS PARA TERCEIROS,REPRODUÇÃO DE MATERIAIS GRAVADOS,COQUERIAS,FABRICAÇÃO DE PRODUTOS DERIVADOS DO PETRÓLEO,ELABORAÇÃO DE COMBUSTÍVEIS NUCLEARES,FABRICAÇÃO DE PRODUTOS QUÍMICOS INORGÂNICOS,FABRICAÇÃO DE PRODUTOS QUÍMICOS ORGÂNICOS,FABRICAÇÃO DE RESINAS E ELASTÔMEROS,"FABRICAÇÃO DE FIBRAS, FIOS, CABOS E FILAMENTOS CONTÍNUOS ARTIFICIAIS E SINTÉTICOS",FABRICAÇÃO DE PRODUTOS FARMACÊUTICOS,FABRICAÇÃO DE DEFENSIVOS AGRÍCOLAS,"FABRICAÇÃO DE SABÕES, DETERGENTES, PRODUTOS DE LIMPEZA E ARTIGOS DE PERFUMARIA","FABRICAÇÃO DE TINTAS, VERNIZES, ESMALTES, LACAS E PRODUTOS AFINS",FABRICAÇÃO DE PRODUTOS E PREPARADOS QUÍMICOS DIVERSOS,FABRICAÇÃO DE ARTIGOS DE BORRACHA,FABRICAÇÃO DE PRODUTOS DE PLÁSTICO,FABRICAÇÃO DE VIDRO E DE PRODUTOS DO VIDRO,FABRICAÇÃO DE CIMENTO,"FABRICAÇÃO DE ARTEFATOS DE CONCRETO, CIMENTO, FIBROCIMENTO, GESSO E ESTUQUE",FABRICAÇÃO DE PRODUTOS CERÂMICOS,APARELHAMENTO DE PEDRAS E FABRICAÇÃO DE CAL E DE OUTROS PRODUTOS DE MINERAIS NÃOMETÁLICOS,PRODUÇÃO DE FERROGUSA E DE FERROLIGAS,SIDERURGIA,FABRICAÇÃO DE TUBOS EXCETO EM SIDERÚRGICAS,METALURGIA DE METAIS NÃOFERROSOS,FUNDIÇÃO,FABRICAÇÃO DE ESTRUTURAS METÁLICAS E OBRAS DE CALDEIRARIA PESADA,"FABRICAÇÃO DE TANQUES, CALDEIRAS E RESERVATÓRIOS METÁLICOS","FORJARIA, ESTAMPARIA, METALURGIA DO PÓ E SERVIÇOS DE TRATAMENTO DE METAIS","FABRICAÇÃO DE ARTIGOS DE CUTELARIA, DE SERRALHERIA E FERRAMENTAS MANUAIS","MANUTENÇÃO E REPARAÇÃO DE TANQUES, CALDEIRAS E RESERVATÓRIOS METÁLICOS",FABRICAÇÃO DE PRODUTOS DIVERSOS DE METAL,"FABRICAÇÃO DE MOTORES, BOMBAS, COMPRESSORES E EQUIPAMENTOS DE TRANSMISSÃO",FABRICAÇÃO DE MÁQUINAS E EQUIPAMENTOS DE USO GERAL,"FABRICAÇÃO DE TRATORES E DE MÁQUINAS E EQUIPAMENTOS PARA A AGRICULTURA, AVICULTURA E OBTENÇÃO DE PRODUTOS ANIMAIS",FABRICAÇÃO DE MÁQUINASFERRAMENTA,FABRICAÇÃO DE MÁQUINAS E EQUIPAMENTOS DE USO NA EXTRAÇÃO MINERAL E CONSTRUÇÃO,FABRICAÇÃO DE OUTRAS MÁQUINAS E EQUIPAMENTOS DE USO ESPECÍFICO,"FABRICAÇÃO DE ARMAS, MUNIÇÕES E EQUIPAMENTOS MILITARES",FABRICAÇÃO DE ELETRODOMÉSTI

## Correlacao de Pearson


In [23]:
df["faixa_remuneracao_media_sm"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [ ]:
df_primeiras_faixas=df[df['faixa_remuneracao_media_sm'].isin([0,1,2,3])] 
df_faixas_meio= df[df['faixa_remuneracao_media_sm'].isin([4,5,6,7])] 
df_ultimas_faixas= df[df['faixa_remuneracao_media_sm'].isin([ 8,  9, 10, 11])] 
dataframes = {
    "df_primeiras_faixas": df_primeiras_faixas,
    "df_faixas_meio": df_faixas_meio,
    "df_ultimas_faixas": df_ultimas_faixas,
    
}

In [ ]:
dataframes = {
    "df_primeiras_faixas": df_primeiras_faixas,
    "df_faixas_meio": df_faixas_meio,
    "df_ultimas_faixas": df_ultimas_faixas,
    
}

In [ ]:
df_primeiras_faixas=df[df['faixa_remuneracao_media_sm'].isin([0,1,2,3])] 
df_faixas_meio= df[df['faixa_remuneracao_media_sm'].isin([4,5,6,7])] 
df_ultimas_faixas= df[df['faixa_remuneracao_media_sm'].isin([ 8,  9, 10, 11])] 
dataframes = {
    "df_primeiras_faixas": df_primeiras_faixas,
    "df_faixas_meio": df_faixas_meio,
    "df_ultimas_faixas": df_ultimas_faixas,
    
}

for nome, dataframe in dataframes.items(): 
    # Calculando a correlação de Pearson de todas as features com a variável alvo
    correlation_matrix = dataframe.corr()

    # Extrair as correlações da variável alvo e descartar a própria variável alvo da série
    target_correlation = correlation_matrix['faixa_remuneracao_media_sm'].drop('faixa_remuneracao_media_sm')

    # Ordenar as correlações em ordem decrescente de valor absoluto para identificar as mais significativas
    top_features_correlation = target_correlation.abs().sort_values(ascending=False)
    print(f"para o {nome} as 30 variaveis mais relevantes de acordo com a correlação de Pearson sao: ")
    # Mostrando as top 10 features mais correlacionadas com a variável alvo
    print(top_features_correlation.head(30))


para o df_primeiras_faixas as 30 variaveis mais relevantes de acordo com a correlação de Pearson sao: 
COMÉRCIO A VAREJO DE COMBUSTÍVEIS                                                            0.511543
ATIVIDADES DE RÁDIO E DE TELEVISÃO                                                           0.486746
30 A 39                                                                                      0.484955
15 A 17                                                                                      0.459110
ATIVIDADES DE INVESTIGAÇÃO, VIGILÂNCIA E SEGURANÇA                                           0.449804
Aprendiz                                                                                     0.442436
40 A 49                                                                                      0.441974
CLT U/ PJ Ind                                                                                0.430163
ATIVIDADES DE ORGANIZAÇÕES SINDICAIS                                             

In [ ]:
##DEPOIS COMPARAR COM O RESULTADO MASCULINO PARA AMBAS AS FAIXAS 

In [28]:
for nome, dataframe in dataframes.items(): 
    X = dataframe.drop('faixa_remuneracao_media_sm', axis=1)
    y = dataframe['faixa_remuneracao_media_sm']

    chi2_features = SelectKBest(chi2, k=10)
    X_kbest_features = chi2_features.fit_transform(X, y)

    # Obtendo os scores para as features selecionadas novamente
    chi2_scores = chi2_features.scores_

    # Criando um dataframe para visualizar os scores de cada feature
    chi2_scores_df = pd.DataFrame(chi2_scores, index=X.columns, columns=["Chi2 Score"]).sort_values(by="Chi2 Score", ascending=False)
    print(f"para o {nome} as 10 variaveis mais relevantes de acordo com o Chi2 Score sao: ")
    print(chi2_scores_df.head(10))
    


para o df_primeiras_faixas as 10 variaveis mais relevantes de acordo com a correlação de Pearson sao: 
                                                       Chi2 Score
CLT U/ PJ Ind                                       273057.057766
30 A 39                                             219112.420245
ESTABELECIMENTOS HOTELEIROS E OUTROS TIPOS DE A...  190655.565565
COMÉRCIO A VAREJO E POR ATACADO DE PEÇAS E ACES...  183168.397453
Comércio Varejista                                  181852.326060
Aprendiz                                            166962.372150
18 A 24                                             166112.481898
COMÉRCIO VAREJISTA NÃO ESPECIALIZADO                143073.019542
Superior Completo                                   139153.449710
40 A 49                                             138245.690134
para o df_faixas_meio as 10 variaveis mais relevantes de acordo com a correlação de Pearson sao: 
                                                       Chi2 Score
RESTAUR

Para as Primeiras Faixas Salariais:
Tipos de Contratação e Faixa Etária: Observa-se uma forte presença de categorias relacionadas a tipos de contratação (como "CLT U/ PJ Ind") e faixas etárias ("30 A 39", "18 A 24", "40 A 49"), sugerindo que esses fatores têm uma influência significativa nas primeiras faixas salariais.
Setores Específicos: Há uma presença marcante de setores como estabelecimentos hoteleiros, comércio de peças e acessórios para veículos automotores e comércio varejista, indicando a importância desses setores para empregos nas faixas salariais mais baixas.
Para as Faixas Médias de Salários:
Setor de Serviços: Destaca-se a importância do setor de serviços, como restaurantes e outros estabelecimentos de alimentação, além do comércio varejista de tecidos e artigos de armarinho. Isso mostra que o setor de serviços tem um papel relevante nas faixas salariais intermediárias.
Perfil Educacional e Tipos de Contratação: A presença de variáveis como "Alimentos e Bebidas" e "Até 5ª Incompleto" aponta para a relevância do nível educacional e do setor alimentício nestas faixas salariais, além de diferentes modalidades de contratação.
Para as Últimas Faixas Salariais:
Setor Público: Nota-se uma forte correlação com o setor público ("Estatutário não Efetivo", "Administração Pública", "Estatutário"), sugerindo que cargos públicos estatutários e a administração pública estão fortemente associados às faixas salariais mais altas.
Educação e Construção Civil: Há uma relevância dos setores de educação profissional e construção civil, além da faixa etária "25 A 29" e níveis educacionais como "Fundamental Completo" e "6ª a 9ª Fundamental", indicando diversidade nos perfis profissionais e educacionais nas faixas salariais superiores.
Conclusões Gerais:
Diversidade Setorial e de Perfil: A diversidade nos setores e perfis profissionais/educacionais associados a cada segmento de faixa salarial destaca a complexidade do mercado de trabalho e a influência de vários fatores na determinação da remuneração.
Influência da Educação e Tipo de Contratação: O nível educacional e o tipo de contratação parecem desempenhar papéis significativos em todas as faixas, sugerindo pontos de atenção para políticas de emprego e educação.
Setor Público nas Faixas Superiores: A forte associação do setor público com as faixas salariais mais altas pode refletir questões como estabilidade e benefícios associados a esses cargos.
Essas análises oferecem uma visão valiosa sobre os fatores que influenciam as remunerações em diferentes níveis e podem ser usadas para direcionar estudos mais detalhados, políticas de RH e estratégias de desenvolvimento profissional.

## RFE

In [31]:

for nome, dataframe in dataframes.items(): 
    X = dataframe.drop('faixa_remuneracao_media_sm', axis=1)
    y = dataframe['faixa_remuneracao_media_sm']
    model = LinearRegression()
    rfe = RFE(estimator=model, n_features_to_select=10)
    rfe.fit(X, y)
    selected_features_rfe = X.columns[rfe.support_]
    print(f"para o {nome} as 10 variaveis mais relevantes de acordo com o rfe sao: ")
    print(selected_features_rfe)

para o df_primeiras_faixas as 10 variaveis mais relevantes de acordo com o rfe sao: 
Index(['ATIVIDADES DE SERVIÇOS RELACIONADOS COM A EXTRAÇÃO DE PETRÓLEO E GÁS  EXCETO A PROSPECÇÃO REALIZADA POR TERCEIROS',
       'EXTRAÇÃO DE MINÉRIO DE FERRO',
       'FABRICAÇÃO DE EQUIPAMENTOS PARA DISTRIBUIÇÃO E CONTROLE DE ENERGIA ELÉTRICA',
       'FABRICAÇÃO DE FIOS, CABOS E CONDUTORES ELÉTRICOS ISOLADOS',
       'FABRICAÇÃO DE PILHAS, BATERIAS E ACUMULADORES ELÉTRICOS',
       'COMÉRCIO ATACADISTA DE MATÉRIAS PRIMAS AGRÍCOLAS, ANIMAIS VIVOS',
       'ALUGUEL DE OBJETOS PESSOAIS E DOMÉSTICOS',
       'PESQUISA E DESENVOLVIMENTO DAS CIÊNCIAS FÍSICAS E NATURAIS',
       'ATIVIDADES DE ORGANIZAÇÕES SINDICAIS', 'Avulso'],
      dtype='object')
para o df_faixas_meio as 10 variaveis mais relevantes de acordo com o rfe sao: 
Index(['DESDOBRAMENTO DE MADEIRA', 'FABRICAÇÃO DE RESINAS E ELASTÔMEROS',
       'FUNDIÇÃO', 'FABRICAÇÃO DE PILHAS, BATERIAS E ACUMULADORES ELÉTRICOS',
       'FABRICAÇÃO DE MATE

## lasso 

In [34]:
from sklearn.linear_model import LassoCV
for nome, dataframe in dataframes.items(): 
    X = dataframe.drop('faixa_remuneracao_media_sm', axis=1)
    y = dataframe['faixa_remuneracao_media_sm']
    lasso = LassoCV(cv=5).fit(X, y)
    important_features_lasso = np.where(lasso.coef_ != 0)[0]
    print(f"para o {nome} as 10 variaveis mais relevantes de acordo com o rfe sao: ")
    print(X.columns[important_features_lasso])

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.6489538352967301, tolerance: 0.17274971056439964
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.31193003955413445, tolerance: 0.17274971056439964
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.25457759265866287, tolerance: 0.17274971056439964
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent

para o df_primeiras_faixas as 10 variaveis mais relevantes de acordo com o rfe sao: 
Index(['FABRICAÇÃO DE OUTROS PRODUTOS ALIMENTÍCIOS',
       'CONFECÇÃO DE ARTIGOS DO VESTUÁRIO',
       'IMPRESSÃO E SERVIÇOS CONEXOS PARA TERCEIROS',
       'FABRICAÇÃO DE PRODUTOS FARMACÊUTICOS',
       'FABRICAÇÃO DE PRODUTOS DE PLÁSTICO', 'FABRICAÇÃO DE ELETRODOMÉSTICOS',
       'FABRICAÇÃO DE FIOS, CABOS E CONDUTORES ELÉTRICOS ISOLADOS',
       'FABRICAÇÃO DE ARTIGOS DO MOBILIÁRIO',
       'CAPTAÇÃO, TRATAMENTO E DISTRIBUIÇÃO DE ÁGUA', 'PREPARAÇÃO DO TERRENO',
       'CONSTRUÇÃO DE EDIFÍCIOS E OBRAS DE ENGENHARIA  CIVIL',
       'COMÉRCIO A VAREJO E POR ATACADO DE VEÍCULOS AUTOMOTORES',
       'MANUTENÇÃO E REPARAÇÃO DE VEÍCULOS AUTOMOTORES',
       'COMÉRCIO A VAREJO DE COMBUSTÍVEIS',
       'REPRESENTANTES COMERCIAIS E AGENTES DO COMÉRCIO',
       'COMÉRCIO ATACADISTA DE MÁQUINAS, APARELHOS E EQUIPAMENTOS PARA USOS AGROPECUÁRIO, COMERCIAL, DE ESCRITÓRIO, INDUSTRIAL, TÉCNICO E PROFISSIONAL',
    

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.18435974233189256, tolerance: 0.17274971056439958
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.34045977972577646, tolerance: 0.17274971056439958
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.23668447570926787, tolerance: 0.17274971056439958
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descen

para o df_faixas_meio as 10 variaveis mais relevantes de acordo com o rfe sao: 
Index(['FABRICAÇÃO DE BEBIDAS', 'IMPRESSÃO E SERVIÇOS CONEXOS PARA TERCEIROS',
       'FABRICAÇÃO DE PRODUTOS QUÍMICOS INORGÂNICOS',
       'FABRICAÇÃO DE PRODUTOS DIVERSOS DE METAL',
       'FABRICAÇÃO DE APARELHOS RECEPTORES DE RÁDIO E TELEVISÃO E DE REPRODUÇÃO, GRAVAÇÃO OU AMPLIFICAÇÃO DE SOM E VÍDEO',
       'FABRICAÇÃO DE PEÇAS E ACESSÓRIOS PARA VEÍCULOS AUTOMOTORES',
       'FABRICAÇÃO DE OUTROS EQUIPAMENTOS DE TRANSPORTE',
       'CAPTAÇÃO, TRATAMENTO E DISTRIBUIÇÃO DE ÁGUA',
       'CONSTRUÇÃO DE EDIFÍCIOS E OBRAS DE ENGENHARIA  CIVIL',
       'OBRAS DE INSTALAÇÕES',
       'COMÉRCIO A VAREJO E POR ATACADO DE VEÍCULOS AUTOMOTORES',
       'MANUTENÇÃO E REPARAÇÃO DE VEÍCULOS AUTOMOTORES',
       'COMÉRCIO A VAREJO DE COMBUSTÍVEIS',
       'COMÉRCIO ATACADISTA DE MATÉRIAS PRIMAS AGRÍCOLAS, ANIMAIS VIVOS',
       'COMÉRCIO ATACADISTA DE PRODUTOS ALIMENTÍCIOS, BEBIDAS E FUMO',
       'COMÉRCIO ATACADIST

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.2280622321864314, tolerance: 0.17274971056439964
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.9652104510694244, tolerance: 0.17274971056439964
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.7905711728017764, tolerance: 0.17274971056439964
  model = cd_fast.enet_coordinate_descent_gram(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.p

para o df_ultimas_faixas as 10 variaveis mais relevantes de acordo com o rfe sao: 
Index(['EDIÇÃO', 'FABRICAÇÃO DE PRODUTOS DERIVADOS DO PETRÓLEO',
       'FABRICAÇÃO DE PRODUTOS QUÍMICOS ORGÂNICOS',
       'FABRICAÇÃO DE PRODUTOS FARMACÊUTICOS', 'SIDERURGIA',
       'PRODUÇÃO E DISTRIBUIÇÃO DE ENERGIA ELÉTRICA',
       'COMÉRCIO A VAREJO E POR ATACADO DE VEÍCULOS AUTOMOTORES',
       'COMÉRCIO ATACADISTA DE PRODUTOS ALIMENTÍCIOS, BEBIDAS E FUMO',
       'COMÉRCIO ATACADISTA DE ARTIGOS DE USOS PESSOAL E DOMÉSTICO',
       'COMÉRCIO ATACADISTA DE MÁQUINAS, APARELHOS E EQUIPAMENTOS PARA USOS AGROPECUÁRIO, COMERCIAL, DE ESCRITÓRIO, INDUSTRIAL, TÉCNICO E PROFISSIONAL',
       'COMÉRCIO VAREJISTA NÃO ESPECIALIZADO',
       'COMÉRCIO VAREJISTA DE OUTROS PRODUTOS',
       'OUTROS TRANSPORTES TERRESTRES',
       'INTERMEDIAÇÃO MONETÁRIA  DEPÓSITOS À VISTA',
       'INTERMEDIAÇÃO NÃO MONETÁRIA  OUTROS TIPOS DE DEPÓSITOS',
       'OUTRAS ATIVIDADES DE INTERMEDIAÇÃO FINANCEIRA, NÃO ESPECIFICADAS 

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.077e+01, tolerance: 2.160e-01
  model = cd_fast.enet_coordinate_descent(


RDE

Primeiras Faixas Salariais
Diversidade Setorial: A presença de setores como extração de minério, fabricação de equipamentos elétricos, e atividades relacionadas à agricultura e pesquisa científica indica uma diversidade significativa de setores que influenciam as primeiras faixas salariais. Isso sugere que, mesmo em faixas salariais mais baixas, há uma variedade de empregos qualificados e tecnicamente sofisticados.
Atividades Específicas: A inclusão de atividades como aluguel de objetos pessoais e domésticos, e organizações sindicais destaca nichos específicos de atividade que podem oferecer oportunidades de emprego ou remuneração nesses níveis.
Faixas Médias de Salários
Indústria e Administração Pública: O foco em setores industriais, como desdobramento de madeira, fabricação de resinas, fundição, e a menção específica à administração pública e ao estatuto, ressalta a importância destas áreas para as faixas salariais intermediárias. A presença de capitais específicas como Macapá e Natal pode indicar a influência regional nas oportunidades de emprego e salários.
Indústria Calçadista: A menção à indústria de calçados sublinha a relevância deste setor para a geração de empregos na faixa salarial média, possivelmente refletindo a importância de setores tradicionais de manufatura.
Últimas Faixas Salariais
Construção e Energia: A seleção de atividades como a fabricação de estruturas metálicas, construção de veículos ferroviários, e comércio varejista de combustíveis aponta para a importância do setor de construção e energia nas faixas salariais mais altas. Isso pode refletir demandas por habilidades especializadas e a importância de infraestrutura e energia para a economia.
Segurança e Capitais Específicas: A inclusão de atividades de investigação, vigilância, e segurança, juntamente com várias capitais (como Boa Vista, Goiânia, entre outras), destaca tanto a relevância do setor de segurança quanto a influência regional nos empregos de faixas salariais mais altas.
Conclusões Gerais
Especialização e Diversificação: Os resultados refletem uma tendência de especialização e diversificação setorial em todas as faixas salariais, indicando que diferentes habilidades e setores têm papéis importantes em cada segmento.
Influência Regional: A presença de capitais específicas nos resultados das faixas médias e últimas sugere uma influência regional significativa nas oportunidades de emprego e nos níveis salariais, sublinhando a importância do contexto geográfico na análise de remuneração.
Oportunidades Setoriais: A análise aponta para oportunidades específicas setoriais que podem ser exploradas para avanços profissionais ou desenvolvimento econômico, dependendo da faixa salarial considerada.

Analise final :

1. Relevância de Setores Específicos:
Em todas as análises, observamos a importância de certos setores para diferentes faixas salariais. Por exemplo, setores como a fabricação (alimentos, produtos químicos, equipamentos elétricos), construção civil e serviços (educação, saúde, alojamento e alimentação) são consistentemente destacados. Isso indica que esses setores têm um impacto significativo na economia e no mercado de trabalho, oferecendo oportunidades em uma ampla gama de faixas salariais.

2. Influência do Nível de Especialização e Qualificação:
As análises também mostram uma tendência de que quanto mais especializada e qualificada é a atividade ou setor, maior é a tendência de estar associada a faixas salariais mais altas. Isso é evidente na ênfase em setores de alta tecnologia, serviços especializados e indústrias que requerem habilidades técnicas avançadas nas faixas salariais superiores.

3. Importância do Comércio e Indústria:
Em todas as análises, o comércio (varejista e atacadista) e a indústria (desde manufatura até setores de alta tecnologia) são destacados como cruciais para a geração de empregos e determinação de faixas salariais. Isso reflete a diversidade econômica e a importância desses setores como motores de crescimento e desenvolvimento.

4. Variáveis Demográficas e de Emprego:
As análises ressaltam a influência de variáveis demográficas (faixas etárias) e de emprego (tipos de contrato, níveis de educação) em todas as faixas salariais. Isso sugere que as políticas de emprego e educação podem desempenhar um papel fundamental na mobilidade salarial e no acesso a oportunidades.

5. Correlação Regional:
Particularmente no RFE e LASSO, a menção a capitais e localizações específicas indica uma correlação regional com as faixas salariais, sugerindo que fatores geográficos e o desenvolvimento econômico regional podem influenciar as oportunidades de emprego e salários.

Conclusão Geral:
A convergência desses resultados aponta para a complexidade do mercado de trabalho e a interação de múltiplos fatores — setoriais, demográficos, educacionais e regionais — na determinação das faixas salariais. Reconhecer essas relações ajuda a entender melhor as dinâmicas de emprego e pode orientar esforços para a capacitação profissional, desenvolvimento econômico e formulação de políticas públicas para promover a igualdade de oportunidades e crescimento sustentável.